In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

In [2]:


# --- Étape 1 : Acquisition et Préparation des Données (Data Engineering) ---

def load_and_prepare_data(file_path):
    """Charge les données, nettoie et effectue le Feature Engineering."""
    print("1. Chargement et préparation des données...")

    # Chargement des données
    df = pd.read_csv(file_path)

    # Renommage des colonnes (pour gérer le BOM et les espaces)
    df.columns = ['UDI', 'Product_ID', 'Type', 'Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 'Machine_failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

    # La colonne 'Machine_failure' est notre cible
    df.rename(columns={'Machine_failure': 'Failure'}, inplace=True)

    # Feature Engineering simple : Calcul de la différence de température
    df['Temp_Diff'] = df['Process_temperature_K'] - df['Air_temperature_K']

    # Suppression des colonnes non nécessaires et des colonnes de défaillance individuelles
    failure_cols = ['TWF', 'HDF', 'PWF', 'OSF', 'RNF']
    df = df.drop(columns=['UDI', 'Product_ID'] + failure_cols)

    # Séparation des features (X) et de la cible (y)
    X = df.drop('Failure', axis=1)
    y = df['Failure']

    print(f"   - Taille du jeu de données après nettoyage : {X.shape}")
    print(f"   - Nombre de défaillances (cible=1) : {y.sum()}")

    return X, y


In [3]:

# --- Étape 2 : Modélisation Machine Learning (Data Science) ---

def train_and_evaluate_model(X, y):
    """Entraîne un modèle de classification et évalue ses performances."""
    print("\n2. Entraînement et évaluation du modèle...")

    # Séparation des données en ensembles d'entraînement et de test
    # Stratify=y est crucial car les défaillances sont rares (classe déséquilibrée)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Définition des colonnes à transformer
    numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    # Création du préprocesseur (StandardScaler pour les numériques, OneHotEncoder pour les catégorielles)
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )

    # Création du pipeline (Préprocesseur + Modèle)
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        # class_weight='balanced' pour gérer le déséquilibre des classes
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
    ])

    # Entraînement du modèle
    model_pipeline.fit(X_train, y_train)

    # Prédiction sur l'ensemble de test
    y_pred = model_pipeline.predict(X_test)

    # Évaluation
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['No Failure', 'Failure'])

    print(f"   - Précision (Accuracy) du modèle : {accuracy:.4f}")
    print("\n   - Rapport de classification :\n", report)

    # Sauvegarde du modèle entraîné (pour l'étape MLOps)
    model_filename = 'predictive_maintenance_model.joblib'
    joblib.dump(model_pipeline, model_filename)
    print(f"\n3. Modèle sauvegardé sous : {model_filename}")

    return model_pipeline

In [6]:


if __name__ == "__main__":
    DATA_FILE = 'ai4i2020.csv'

    if not os.path.exists(DATA_FILE):
        print(f"Erreur : Le fichier de données '{DATA_FILE}' est introuvable.")
        print("Veuillez vous assurer qu'il est dans le répertoire courant.")
    else:
        # Étape 1
        X, y = load_and_prepare_data(DATA_FILE)

        # Étape 2
        trained_model = train_and_evaluate_model(X, y)

        print("\n--- Pipeline d'Entraînement Terminé ---")


1. Chargement et préparation des données...
   - Taille du jeu de données après nettoyage : (10000, 7)
   - Nombre de défaillances (cible=1) : 339

2. Entraînement et évaluation du modèle...
   - Précision (Accuracy) du modèle : 0.9860

   - Rapport de classification :
               precision    recall  f1-score   support

  No Failure       0.99      1.00      0.99      1932
     Failure       0.93      0.63      0.75        68

    accuracy                           0.99      2000
   macro avg       0.96      0.82      0.87      2000
weighted avg       0.99      0.99      0.98      2000


3. Modèle sauvegardé sous : predictive_maintenance_model.joblib

--- Pipeline d'Entraînement Terminé ---
